In [ ]:
# Check for running GPU
!nvidia-smi

In [ ]:
#run once for initial installs
! pip install farm-haystack
! pip install git+https://github.com/deepset-ai/haystack.git
! pip install urllib3==1.25.4
! pip install torch==1.6.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
#Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2
! sleep 30

In [3]:
# Connect to Elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

02/10/2021 03:13:19 - INFO - faiss -   Loading faiss with AVX2 support.
02/10/2021 03:13:19 - INFO - faiss -   Loading faiss.
02/10/2021 03:13:22 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.082s]
02/10/2021 03:13:22 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.007s]
02/10/2021 03:13:22 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.053s]
02/10/2021 03:13:22 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.002s]


In [4]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [45]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader

# Testing with tutorial data
# doc_dir = "data/article_txt_got"
# s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
# fetch_archive_from_http(url=s3_url, output_dir=doc_dir)
# dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

import haystack

converter = haystack.file_converter.txt.TextConverter(
                    remove_numeric_tables=False,
                    valid_languages = ["en"])

as4 = converter.convert(file_path="/content/as4.txt")


processor = haystack.preprocessor.preprocessor.PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="sentence",
    split_length=1,
    split_respect_sentence_boundary=False,
    split_overlap=0
)

as4Docs = processor.process(as4)
# print(as4Docs)

for i in range(0,len(as4Docs)):
    print(i)
    print(":\n")
    print(as4Docs[i])
    print("---------------")

# ! docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.6.2

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(
    host="localhost",
    username="",
    password="",
    index="document"
)

document_store.delete_all_documents(index='document')
document_store.write_documents(as4Docs)

backagain = document_store.get_all_documents();

for i in range(0,len(backagain)):
    print(i)
    print(":\n")
    print(backagain[i])
    print("---------------")

# Connecting point to preprocessing of as4
# document_store.write_documents(dicts)

02/10/2021 03:57:46 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.006s]
02/10/2021 03:57:46 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.006s]
02/10/2021 03:57:46 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.028s]
02/10/2021 03:57:46 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.002s]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0
:

{'text': 'Introduction\nThis booklet presents our Strategies for disease control in spring 2019 for the main\ncombinable crops, put together by NIAB TAG staff from our latest trial results.', 'meta': {'_split_id': 0}}
---------------
1
:

{'text': 'For winter wheat we have continued with a suggested range of fungicide expenditure\naccording to disease risk, itself determined by a range of factors which we encourage\nmembers to assess.', 'meta': {'_split_id': 1}}
---------------
2
:

{'text': 'We have increased the significance of factors such as sowing\ndate and variety choice in this, as these cultural control measures grow in importance.', 'meta': {'_split_id': 2}}
---------------
3
:

{'text': 'For the winter barley crop we have continued our guidance towards three-spray\nprogrammes.', 'meta': {'_split_id': 3}}
---------------
4
:

{'text': 'Any changes in the Strategi

02/10/2021 03:57:47 - INFO - elasticsearch -   POST http://localhost:9200/document/_delete_by_query [status:200 request:0.049s]
02/10/2021 03:57:48 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.757s]
02/10/2021 03:57:48 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=1d&size=10000 [status:200 request:0.047s]
02/10/2021 03:57:48 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.003s]
02/10/2021 03:57:48 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.005s]


0
:

{'text': 'Introduction\nThis booklet presents our Strategies for disease control in spring 2019 for the main\ncombinable crops, put together by NIAB TAG staff from our latest trial results.', 'id': '65a22533-5009-4ec2-a1c7-b319d80229e5', 'score': None, 'probability': None, 'question': None, 'meta': {'_split_id': 0}, 'embedding': None}
---------------
1
:

{'text': 'For winter wheat we have continued with a suggested range of fungicide expenditure\naccording to disease risk, itself determined by a range of factors which we encourage\nmembers to assess.', 'id': '1e7f1180-2940-4444-ac2b-21b76e924046', 'score': None, 'probability': None, 'question': None, 'meta': {'_split_id': 1}, 'embedding': None}
---------------
2
:

{'text': 'We have increased the significance of factors such as sowing\ndate and variety choice in this, as these cultural control measures grow in importance.', 'id': '321e2c9a-4b5c-4973-a837-0ced117c4ca7', 'score': None, 'probability': None, 'question': None, 'meta':

In [46]:
# Fast filter to narrow down text - Default BM25, can be cunstomised
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [47]:
# Alternative Retriever - neural network
from haystack.retriever.dense import DensePassageRetriever
dense_retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)

In [8]:
# Reader to further scan with Hugging Face models
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)
reader = FARMReader(model_name_or_path="deepset/bert-large-uncased-whole-word-masking-squad2", use_gpu=True)

02/10/2021 03:14:27 - INFO - farm.utils -   Using device: CUDA 
02/10/2021 03:14:27 - INFO - farm.utils -   Number of GPUs: 1
02/10/2021 03:14:27 - INFO - farm.utils -   Distributed Training: False
02/10/2021 03:14:27 - INFO - farm.utils -   Automatic Mixed Precision: None
02/10/2021 03:15:57 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='prediction_heads' was already logged with value='TextSimilarityHead' for run ID='76b0a212befb401185db5c777aec8b90'. Attempted logging new value 'QuestionAnsweringHead'.
02/10/2021 03:15:58 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='processor' was already logged with value='TextSimilarityProcessor' for run ID='76b0a212befb401185db5c777aec8b90'. Attempted logging new value 'SquadProcessor'.
02/10/2021 03:15:58 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
02/10/2021 03:15

In [48]:
document_store.update_embeddings(dense_retriever)

02/10/2021 03:58:47 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.012s]
02/10/2021 03:58:47 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for 401 docs ...
02/10/2021 03:58:47 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=1d&size=10000 [status:200 request:0.043s]
02/10/2021 03:58:47 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.007s]
02/10/2021 03:58:47 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.005s]
Creating Embeddings: 100%|██████████| 26/26 [00:06<00:00,  3.98 Batches/s]
02/10/2021 03:58:56 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:2.469s]


In [50]:
from haystack.pipeline import ExtractiveQAPipeline

# Original Finder deprecated, pipeline allows more flexibility
# prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)

# top_k_retriever -> the more retriever the more document scanned in Reader, slower but higher hit rate
extractive_pipeline = ExtractiveQAPipeline(reader=reader, retriever=dense_retriever) # Other options: Document Search, Generative, FAQ

In [51]:
question = 'What should be applied to winter barley in T1?'
prediction = extractive_pipeline.run(query=question, top_k_retriever=2, top_k_reader=2)
# print_answers(prediction, details="all") #details: all, medium, minimal
# data format: {query,'answers':[{'answer','score','probability','context','document_id','offset','meta'}]}
pp.pprint(prediction['answers'])
pp.pprint(retriever.retrieve(query=question,top_k=5))

Creating Embeddings: 100%|██████████| 1/1 [00:00<00:00, 65.39 Batches/s]
02/10/2021 03:59:12 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.022s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 13.33 Batches/s]
02/10/2021 03:59:12 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.020s]


[ { 'answer': 'a low dose of triazole',
    'context': 'Winter oats\n'
               'T0: if any level of crown rust develops before the T1 timing, '
               'a low dose of triazole (e.g.',
    'document_id': 'a568931f-f546-4bbf-b17d-a4238f4b3ec6',
    'meta': {'_split_id': 220},
    'offset_end': 96,
    'offset_end_in_doc': 96,
    'offset_start': 74,
    'offset_start_in_doc': 74,
    'probability': 0.7231775522783926,
    'score': 7.682227611541748},
  { 'answer': 'Autumn sown crops however should be treated as for winter wheat',
    'context': 'Autumn sown crops however should be treated as for winter '
               'wheat (see section 1).',
    'document_id': '3f46ceab-5c20-42de-9084-334c6f3eeeac',
    'meta': {'_split_id': 254},
    'offset_end': 63,
    'offset_end_in_doc': 63,
    'offset_start': 0,
    'offset_start_in_doc': 0,
    'probability': 0.5017822504929182,
    'score': 0.057032257318496704}]
[ {'text': 'Winter barley\nFungicide programmes\nIn light of resp